In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import sys

In [2]:
SCC = pd.read_csv("../../data/SCC.csv")
Louv = pd.read_csv("../../data/Louv.csv")
LabelProp = pd.read_csv("../../data/LabelProp.csv")
Leiden = pd.read_csv("../../data/Leiden.csv")
nodes = pd.read_csv("../../data/nodes.csv")
Between = pd.read_csv("../../data/Between.csv")
PageRank = pd.read_csv("../../data/PageRank.csv")

In [3]:
def community_det_acc(df1, df2):
    df1['Node_ID:ID'] = [int(i.strip('"')) for i in df1['Name'].tolist()]
    df1.drop(columns=['Name'], inplace=True)

    result_df = pd.merge(df1, df2, on='Node_ID:ID', how='inner')

    percentage_breakdown = result_df.groupby(['Community', 'Status:LABEL']).size().unstack(fill_value=0)
    
    percentage_breakdown_percentage = percentage_breakdown.div(percentage_breakdown.sum(axis=1), axis=0) * 100
    
    return percentage_breakdown_percentage

In [4]:
community_det_acc(SCC, nodes)

Status:LABEL,Hyperglycemia,Hypoglycemia,Normal
Community,,,
0,33.248599,39.368729,27.382672


In [5]:
community_det_acc(Louv, nodes).head()

Status:LABEL,Hyperglycemia,Hypoglycemia,Normal
Community,,,
28,24.219725,35.705368,40.074906
118,29.026217,17.602996,53.370787
195,29.411765,33.564014,37.024221
269,28.461538,38.076923,33.461538
336,29.530201,13.422819,57.046980


In [6]:
community_det_acc(LabelProp, nodes).head()

Status:LABEL,Hyperglycemia,Hypoglycemia,Normal
Community,,,
1,27.390476,16.609524,56.000000
6,26.658127,29.006547,44.335326
8,31.533546,20.287540,48.178914
19,27.031019,39.143279,33.825702
28,26.145833,36.458333,37.395833


In [7]:
community_det_acc(Leiden, nodes).head()

Status:LABEL,Hyperglycemia,Hypoglycemia,Normal
Community,,,
0,27.333228,58.247504,14.419268
2,25.685109,57.898139,16.416753
4,32.246377,53.442029,14.311594
10,26.588751,62.892622,10.518627
18,40.439158,47.941446,11.619396


In [3]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from scipy.signal import find_peaks
from scipy.signal import peak_widths
from scipy.signal import peak_prominences
from scipy.integrate import simps

In [4]:
def clean_merge(Louv, nodes):
    Louv['Node_ID:ID'] = [int(i.strip('"')) for i in Louv['Name'].tolist()]
    Louv.drop(columns=['Name'], inplace=True)
    return pd.merge(Louv, nodes, on='Node_ID:ID', how='inner')

In [10]:
Between['Node_ID:ID'] = [int(i.strip('"')) for i in Between['Name'].tolist()]
Between.drop(columns=['Name'], inplace=True)

In [11]:
result_df = pd.merge(Between, result_df, on='Node_ID:ID', how='inner')

NameError: name 'result_df' is not defined

In [5]:
Leiden.rename(columns={'Community':'Leiden'}, inplace=True)
LabelProp.rename(columns={'Community':'LabelProp'}, inplace=True)
Louv.rename(columns={'Community':'Louv'}, inplace=True)
PageRank.rename(columns={'score':'PageRank'}, inplace=True)
Between.rename(columns={'score':'Between'}, inplace=True)

In [6]:
result_df = clean_merge(Leiden, nodes)
result_df = clean_merge(LabelProp, result_df)
result_df = clean_merge(Louv, result_df)
result_df = clean_merge(Between, result_df)
result_df = clean_merge(PageRank, result_df)

In [ ]:
result_df

,score,Node_ID:ID,Louv,LabelProp,Leiden,SpecID,WaveNumber:float,Absorbance:float,Status:LABEL,Widths:float,Prominences:float,GridSlot
0,780612.210631,1000035,573,576,0,210121-2-30,1136.21860,12579.9250,Hyperglycemia,37.657680,8549.4306,"(22, 12)"
1,145508.637599,1000118,573,793,2,210121-2-30,1192.93850,6354.4082,Hyperglycemia,12.435403,410.4478,"(23, 6)"
2,159325.507257,1000185,42038,5517,10,210121-2-30,1238.72440,5826.6846,Hyperglycemia,9.682399,257.8296,"(24, 5)"
3,188984.167622,1000242,573,793,2,210121-2-30,1277.67650,7566.6401,Hyperglycemia,42.064747,2416.4580,"(25, 7)"
4,85364.859372,1000277,573,793,2,210121-2-30,1301.59450,6515.1807,Hyperglycemia,12.781082,524.1089,"(26, 6)"
...,...,...,...,...,...,...,...,...,...,...,...,...
51922,108428.688334,999683,573,793,2,210121-2-30,895.67200,6981.9697,Hyperglycemia,15.441089,1963.9917,"(17, 6)"
51923,219247.103784,999746,573,793,2,210121-2-30,938.72437,6825.0508,Hyperglycemia,21.179879,1766.2593,"(18, 6)"
51924,125683.846762,999841,42038,5517,10,210121-2-30,1003.64470,5274.4521,Hyperglycemia,13.162760,177.3945,"(20, 5)"
51925,125683.846762,999888,42038,5517,10,210121-2-30,1035.76310,5524.3857,Hyperglycemia,22.778603,287.7753,"(20, 5)"


In [7]:
result_df['GridSlot_x'] = 0
result_df['GridSlot_y'] = 0

for i, r in result_df.iterrows():
    result_df['GridSlot_x'][i], result_df['GridSlot_y'][i] = r['GridSlot'].split(',')
    result_df['GridSlot_x'][i] = int(result_df['GridSlot_x'][i].strip('('))
    result_df['GridSlot_y'][i] = int(result_df['GridSlot_y'][i].strip(')'))

/tmp/ipykernel_76080/1595376135.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df['GridSlot_x'][i], result_df['GridSlot_y'][i] = r['GridSlot'].split(',')
/tmp/ipykernel_76080/1595376135.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '(20' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  result_df['GridSlot_x'][i], result_df['GridSlot_y'][i] = r['GridSlot'].split(',')
/tmp/ipykernel_76080/1595376135.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df['GridSlot_x'][i], result_df['GridS

In [ ]:
result_df

,score,Node_ID:ID,Louv,LabelProp,Leiden,SpecID,WaveNumber:float,Absorbance:float,Status:LABEL,Widths:float,Prominences:float,GridSlot_x,GridSlot_y,Bin
0,780612.210631,1000035,573,576,0,210121-2-30,1136.21860,12579.9250,Hyperglycemia,37.657680,8549.4306,22,12,1100-1200
1,145508.637599,1000118,573,793,2,210121-2-30,1192.93850,6354.4082,Hyperglycemia,12.435403,410.4478,23,6,1100-1200
2,159325.507257,1000185,42038,5517,10,210121-2-30,1238.72440,5826.6846,Hyperglycemia,9.682399,257.8296,24,5,1200-1300
3,188984.167622,1000242,573,793,2,210121-2-30,1277.67650,7566.6401,Hyperglycemia,42.064747,2416.4580,25,7,1200-1300
4,85364.859372,1000277,573,793,2,210121-2-30,1301.59450,6515.1807,Hyperglycemia,12.781082,524.1089,26,6,1300-1400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51922,108428.688334,999683,573,793,2,210121-2-30,895.67200,6981.9697,Hyperglycemia,15.441089,1963.9917,17,6,800-900
51923,219247.103784,999746,573,793,2,210121-2-30,938.72437,6825.0508,Hyperglycemia,21.179879,1766.2593,18,6,900-1000
51924,125683.846762,999841,42038,5517,10,210121-2-30,1003.64470,5274.4521,Hyperglycemia,13.162760,177.3945,20,5,1000-1100
51925,125683.846762,999888,42038,5517,10,210121-2-30,1035.76310,5524.3857,Hyperglycemia,22.778603,287.7753,20,5,1000-1100


In [8]:
result_df.drop(columns={"GridSlot"}, inplace=True)

In [9]:
peaks_df = result_df

In [61]:
from sklearn.preprocessing import MinMaxScaler

# Assuming df is your DataFrame and 'column_name' is the column you want to normalize
column_name = 'PageRank'

scaler = MinMaxScaler()
peaks_df[[column_name]] = scaler.fit_transform(peaks_df[[column_name]])

In [76]:
column_name = 'Widths:float'

scaler = MinMaxScaler()
peaks_df[[column_name]] = scaler.fit_transform(peaks_df[[column_name]])

In [63]:
# Define a function to calculate the bin for a given wavenumber with a specified bin size
def calculate_bin_interval(wavenumber, bin_size):
    bin_start = int((wavenumber - 200) / bin_size) * bin_size + 200
    bin_end = bin_start + bin_size
    return f"{bin_start}-{bin_end}"

# Set the bin size
bin_size = 100

# Add a "Bin" column to the DataFrame
peaks_df['Bin'] = peaks_df['WaveNumber:float'].apply(lambda x: calculate_bin_interval(x, bin_size))

In [77]:
# Pivot table with 'Absorbance', 'PeakWidths', and 'PeakProminences' as values
peak_bins = peaks_df.pivot_table(index='SpecID', columns='Bin', values=['Widths:float', 'PageRank', 'Leiden', 'Between', 'LabelProp'], aggfunc='mean')
peak_bins.columns = [f"{col[0]}_{col[1]}" for col in peak_bins.columns]  # Combine column names
peak_bins.reset_index(inplace=True)

# Merge with 'Status' information
statuses = peaks_df[['SpecID', 'Status:LABEL']].drop_duplicates()
peak_bins = pd.merge(peak_bins, statuses, on='SpecID')

# Set 'SpecID' as the index
peak_bins.set_index('SpecID', inplace=True)

# Fill NaN values with False
peak_bins.fillna(False, inplace=True)

/tmp/ipykernel_76080/1544707029.py:14: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'False' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  peak_bins.fillna(False, inplace=True)


In [78]:
# Splitting the dataframe into features (X) and target variable (y)
X = peak_bins.drop(['Status:LABEL'], axis=1)
y = peak_bins['Status:LABEL']

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

In [79]:
X.head()

,Between_1000-1100,Between_1100-1200,Between_1200-1300,Between_1300-1400,Between_1400-1500,Between_1500-1600,Between_1600-1700,Between_1700-1800,Between_1800-1900,Between_1900-2000,...,Widths:float_1800-1900,Widths:float_1900-2000,Widths:float_200-300,Widths:float_300-400,Widths:float_400-500,Widths:float_500-600,Widths:float_600-700,Widths:float_700-800,Widths:float_800-900,Widths:float_900-1000
SpecID,,,,,,,,,,,,,,,,,,,,,
201210-1-00,False,False,False,False,False,0.000124,False,False,False,False,...,False,False,False,False,False,False,0.027845,False,False,0.042921
201210-1-01,False,0.000599,0.00058,False,0.000648,0.000766,0.00073,False,False,False,...,False,False,False,False,False,0.000557,0.023936,False,0.034161,0.010699
201210-1-02,0.001449,0.001967,0.001263,0.001226,0.001436,False,0.000707,0.01029,False,False,...,False,False,False,False,0.000705,False,0.013644,False,False,0.014281
201210-1-03,0.001186,0.001617,0.001749,False,0.00134,0.002626,0.000705,False,False,False,...,False,False,0.00758,False,0.007039,False,0.018623,False,False,0.025602
201210-1-04,0.000397,0.000253,0.000235,False,0.000306,0.000348,0.000184,False,False,False,...,False,False,False,False,0.006819,False,0.023122,False,False,False


In [80]:
rf = RandomForestClassifier(random_state=1234)
rf.fit(X_train, y_train)

# Initialising and fitting the Extra Trees classifier
et = ExtraTreesClassifier(random_state=1234)
et.fit(X_train, y_train)

ExtraTreesClassifier(random_state=1234)

In [ ]:
def get_feature_importances(model, X):

    # Get feature importances
    feature_importances = model.feature_importances_

    # Creating a DataFrame to display feature importances
    feature_importance_df = pd.DataFrame({'Feature': X.columns, 'Importance': feature_importances})

    # Sorting the DataFrame by importance in descending order
    feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

    # Show the top 10 most important features
    top_10_features = feature_importance_df.head(20)

    return top_10_features

In [81]:
def calculate_metrics(y_test, y_pred):

    # Calculate overall accuracy
    overall_accuracy = accuracy_score(y_test, y_pred)
    print(f"Overall Accuracy: {overall_accuracy}\n")

    # Calculate precision, recall, and F1-score for each class
    report = classification_report(y_test, y_pred)
    print("\nClassification Report:")
    print(report)

    # Show the confusion matrix
    conf_matrix = confusion_matrix(y_test, y_pred)
    print("\nConfusion Matrix:")
    print(conf_matrix)

In [82]:
# Making predictions on the test set
y_pred = rf.predict(X_test)

# Calculating and printing evaluation metrics
calculate_metrics(y_test, y_pred)

Overall Accuracy: 0.7550335570469798


Classification Report:
               precision    recall  f1-score   support

Hyperglycemia       0.76      0.73      0.74       195
 Hypoglycemia       0.75      0.76      0.76       196
       Normal       0.75      0.78      0.76       205

     accuracy                           0.76       596
    macro avg       0.76      0.75      0.75       596
 weighted avg       0.76      0.76      0.75       596


Confusion Matrix:
[[142  24  29]
 [ 24 149  23]
 [ 21  25 159]]


In [83]:
# Making predictions on the test set
y_pred = et.predict(X_test)

# Calculating and printing evaluation metrics
calculate_metrics(y_test, y_pred)

Overall Accuracy: 0.7466442953020134


Classification Report:
               precision    recall  f1-score   support

Hyperglycemia       0.75      0.76      0.76       195
 Hypoglycemia       0.74      0.72      0.73       196
       Normal       0.75      0.76      0.75       205

     accuracy                           0.75       596
    macro avg       0.75      0.75      0.75       596
 weighted avg       0.75      0.75      0.75       596


Confusion Matrix:
[[149  25  21]
 [ 24 141  31]
 [ 25  25 155]]


In [85]:
get_feature_importances(rf, X)

,Feature,Importance
82,Widths:float_200-300,0.046962
88,Widths:float_800-900,0.030174
64,PageRank_200-300,0.026453
74,Widths:float_1200-1300,0.026032
10,Between_200-300,0.024713
72,Widths:float_1000-1100,0.023778
87,Widths:float_700-800,0.021924
73,Widths:float_1100-1200,0.020142
75,Widths:float_1300-1400,0.019912
18,LabelProp_1000-1100,0.018427


In [86]:
get_feature_importances(et, X)

,Feature,Importance
82,Widths:float_200-300,0.033335
64,PageRank_200-300,0.023790
54,PageRank_1000-1100,0.021578
38,Leiden_1200-1300,0.020939
69,PageRank_700-800,0.020724
46,Leiden_200-300,0.020582
75,Widths:float_1300-1400,0.020116
28,LabelProp_200-300,0.019951
59,PageRank_1500-1600,0.019950
40,Leiden_1400-1500,0.019345
